In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input

import tfcoreml

In [2]:
keras_model = Sequential([
    Dense(10, input_shape=(3,)),
    Dense(3, activation='softmax')
])

keras_model.save('./model.h5')

In [3]:
print(keras_model.input_shape)

# get input, output node names for the TF graph from the Keras model
input_name = keras_model.inputs[0].name.split(':')[0]
keras_output_node_name = keras_model.outputs[0].name.split(':')[0]
graph_output_node_name = keras_output_node_name.split('/')[-1]

print(input_name)
print(keras_output_node_name)
print(graph_output_node_name)

core_model = tfcoreml.convert(tf_model_path='./model.h5',
                         input_name_shape_dict={input_name: (1, 3)},
                         output_feature_names=[graph_output_node_name],
                         minimum_ios_deployment_target='13')
core_model.save('./model.mlmodel')

core_model

(None, 3)
dense_input
dense_1/Identity
Identity


0 assert nodes deleted
['sequential/dense/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense/MatMul/ReadVariableOp/resource:0', 'sequential/dense_1/BiasAdd/ReadVariableOp:0', 'sequential/dense_1/MatMul/ReadVariableOp/resource:0', 'sequential/dense_1/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense_1/MatMul/ReadVariableOp:0', 'sequential/dense/MatMul/ReadVariableOp:0', 'sequential/dense/BiasAdd/ReadVariableOp:0']
4 nodes deleted
0 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 4 nodes.
2 identity nodes deleted
2 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/4] Converting op type: 'Placeholder', name: 'dense_input', output_shape: (1, 3).
[SSAConverter] [2/4] Converting op type: 'MatMul', name: 'sequential/dense/MatMul', output_shape: (1, 10).
[SSAConverter] [3/4] Converting op type: 'MatMul', name: 'sequential/dense_1/MatMul', output_shape: (1, 3).
[SSAConverter] [4/4] Converting op type: 'Softmax', name: 'Identity', o

input {
  name: "dense_input"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      dataType: DOUBLE
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: DOUBLE
    }
  }
}
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.1"
  }
}

In [4]:
spec = core_model.get_spec()
spec

specificationVersion: 4
description {
  input {
    name: "dense_input"
    type {
      multiArrayType {
        shape: 1
        shape: 3
        dataType: DOUBLE
      }
    }
  }
  output {
    name: "Identity"
    type {
      multiArrayType {
        dataType: DOUBLE
      }
    }
  }
  metadata {
    userDefined {
      key: "coremltoolsVersion"
      value: "3.1"
    }
  }
}
neuralNetwork {
  layers {
    name: "sequential/dense/MatMul"
    input: "dense_input"
    output: "sequential/dense/MatMul"
    inputTensor {
      rank: 2
      dimValue: 1
      dimValue: 3
    }
    outputTensor {
      rank: 2
      dimValue: 1
      dimValue: 10
    }
    batchedMatmul {
      weightMatrixFirstDimension: 3
      weightMatrixSecondDimension: 10
      hasBias: true
      weights {
        floatValue: -0.2486245036125183
        floatValue: 0.5589599609375
        floatValue: -0.1447615623474121
        floatValue: -0.19452276825904846
        floatValue: 0.19196957349777222
        flo

In [10]:
data = {'dense_input': [[1.0, 1.0, 1.0]]}
predictions = core_model.predict(data)

RuntimeError: {
    NSLocalizedDescription = "Required input feature not passed to neural network.";
}